# Pulser Calculation Example

Following are the imports required to run this example:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import qse
import pulser

## 2D square lattice

In [ ]:
# Parameters in rad/µs and ns
Omega_max = 2.0 * 2 * np.pi
U = Omega_max / 2.0
delta_0 = -6 * U
delta_f = 2 * U
t_rise = 252
t_fall = 500
t_sweep = (delta_f - delta_0) / (2 * np.pi * 10) * 1000

# We extract the reference blockade radius from pulser here
R_interatomic = pulser.devices.MockDevice.rydberg_blockade_radius(U)

L1, L2 = 3, 2  # lattice size

# Now we generate the qbits object that represents 1D lattice with r0 as lattice spacing
# Keeping r0 a bit below blockade radius keeps the nearest neighbours antiferromagnetic
r0 = 0.8 * R_interatomic
q2d = qse.lattices.square(r0, L1, L2)

q2d.draw(radius=R_interatomic)

## Hamiltonian creation for 2D lattice

In [ ]:
# up ramp, constant, downramp waveform
amplitude_afm = pulser.CompositeWaveform(
    pulser.waveforms.RampWaveform(t_rise, 0.0, Omega_max),
    pulser.waveforms.ConstantWaveform(t_sweep, Omega_max),
    pulser.waveforms.RampWaveform(t_fall, Omega_max, 0.0),
)

# corresponding waveform for detuning
detuning_afm = pulser.CompositeWaveform(
    pulser.waveforms.ConstantWaveform(t_rise, delta_0),
    pulser.waveforms.RampWaveform(t_sweep, delta_0, delta_f),
    pulser.waveforms.ConstantWaveform(t_fall, delta_f),
)

In [ ]:
amplitude_afm.draw()
detuning_afm.draw()

## Setting up the calculator

In [ ]:
assert isinstance(amplitude_afm, pulser.waveforms.Waveform)
pcalc = qse.calc.Pulser(qbits=q2d, amplitude=amplitude_afm, detuning=detuning_afm)

## Running the job

In [ ]:
pcalc.build_sequence()
pcalc.calculate()

In [ ]:
pcalc.spins

In [ ]:
pcalc.get_sij()

In [ ]:
pcalc.structure_factor_from_sij(L1, L2, 1)

## Sampling the result

In [ ]:
count = pcalc.results.sample_final_state()

most_freq = {k: v for k, v in count.items() if v > 10}
plt.bar(list(most_freq.keys()), list(most_freq.values()))
plt.xticks(rotation="vertical")
plt.show()

## Version

In [ ]:
!python --version

In [ ]:
print(qse.__name__, qse.__version__)